
<h1 id="Technician-Routing-and-Scheduling-Problem">Technician Routing and Scheduling Problem<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Technician-Routing-and-Scheduling-Problem">¶</a></h1>



<h2 id="Objective-and-Prerequisites">Objective and Prerequisites<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Objective-and-Prerequisites">¶</a></h2><p>In this Jupyter Notebook, you will learn how to formulate a multi-depot vehicle routing problem with time windows constraints using the Gurobi Python API. To fully understand the content of this notebook, the reader should be familiar with object-oriented-programming.</p>
<p>This modeling example is at the intermediate level, where we assume that you know Python and are familiar with the Gurobi Python API. In addition, you have some knowledge about building mathematical optimization models.</p>
<p><strong>Download the Repository</strong> <br/> 
You can download the repository containing this and other examples by clicking <a href="https://github.com/Gurobi/modeling-examples/archive/master.zip">here</a>.</p>
<p><strong>Gurobi License</strong> <br/> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an <a href="https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-TRA-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_technician-routing-sche-problem_COM_EVAL_GITHUB_&amp;utm_term=technician-routing-scheduling-problem&amp;utm_content=C_JPM">evaluation license</a> as a <em>commercial user</em>, or download a <a href="https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-TRA-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_technician-routing-sch-problem_ACADEMIC_EVAL_GITHUB_&amp;utm_term=technician-routing-scheduling-problem&amp;utm_content=C_JPM">free license</a> as an <em>academic user</em>.</p>



<h2 id="Motivation">Motivation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Motivation">¶</a></h2><p>The Technician Routing and Scheduling (TRS) is a common problem faced by telecommunication firms, which must be able to provide services for a large number of customers. With its limited resources, these firms must try to deliver timely, affordable and reliable services to maximize customer satisfaction. The TRS problem involves assignment, scheduling, and routing of multi-skilled technicians to serve customers with unique priorities, service time windows, processing times, skills requirements, and geographical locations. 
In the current practice, these decisions are often being made by operators via heuristics, sometimes manually -and this method
can not only be time-consuming, 
but may also deviate from optima significantly. 
Using mathematical optimization to solve the
TRS problem  will
enable 
telecom operators and dispatchers to automate and improve their assignment, scheduling, and routing decisions, while achieving high customer satisfaction.</p>



<h2 id="Problem-Description">Problem Description<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Problem-Description">¶</a></h2><p>A telecom firm operates multiple service centers to serve its customers. 
Each service center has 
its technicians who are dispatched from the service center to work on their assigned jobs, and return to the center after all the jobs are completed. 
A technician has multiple skills and available working capacity that cannot be exceeded during the scheduling horizon. 
A service order/job has 
a known processing time, 
a customer-specified time window for starting the service, 
a deadline of completing the service, 
and its skill requirements. 
Depending on the nature of the job (routine maintenance or emergency), and relationship with the customer (existing, new), the firm assesses the importance of the job and assigns a priority score to it. 
A job is assigned to at most one technician who possesses the required 
skills, 
but the technician's available capacity during the scheduling horizon cannot be exceeded.
To solve the  TRS problem, telecommunications companies must be able to
simultaneously makes three types of decisions: (i) the assignment of jobs to a technician at all the service centers; 
(ii) the routing of each technician, i.e. the sequence/order of customers for a technician to visit; 
(iii) and the scheduling of jobs, i.e. the timing for a technician to arrive at a customer location and complete the corresponding job. 
The firm’s goal is to minimize the total weighted lateness of all  jobs, with their priority being the weights.</p>
<p>The following constraints must be satisfied:</p>
<ul>
<li>A technician, if utilized, departs from the service center where he/she is based and returns to the same service center after his/her assigned jobs are completed.</li>
<li>A technician’s available capacity during the scheduling horizon cannot be exceeded.</li>
<li>A job, if selected, is assigned to at most one technician who  possesses the required skills.</li>
<li>A technician must arrive at a customer location during an interval (time window) specified by the customer, and must complete a job before the deadline required by the customer. This an important constraint for guaranteeing customer satisfaction. </li>
</ul>
<p>The above basic TRS is a variant of the multi-depot vehicle routing problem with time windows, known as the MDVRPTW in the literature [1].</p>



<h2 id="Solution-Approach">Solution Approach<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Solution-Approach">¶</a></h2><p>Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.</p>
<p>A mathematical optimization model has five components, namely:</p>
<ul>
<li>Sets and indices.</li>
<li>Parameters.</li>
<li>Decision variables.</li>
<li>Objective function(s).</li>
<li>Constraints.</li>
</ul>
<p>To simplify the MIP formulation, we consider that for each job's skill requirements, we can determine the subset of technicians that are qualified to perform the job based on the skills that the technicians have.</p>
<p>We now present a MIP formulation for the basic TRS problem.</p>



<h2 id="Model-Formulation">Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Model-Formulation">¶</a></h2>



<h3 id="Sets-and-Indices">Sets and Indices<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Sets-and-Indices">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-1-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1" style="width: 10.896em; display: inline-block;"><span style="display: inline-block; position: relative; width: 9.051em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1008.99em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-2"><span class="mi" id="MathJax-Span-3" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-4" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-5" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-6" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-7" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-8" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-9" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-10" style="font-family: MathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-11" style="font-family: MathJax_Main;">,</span><span class="mo" id="MathJax-Span-12" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-13" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-14" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-15" style="font-family: MathJax_Main; padding-left: 0.182em;">,</span><span class="mi" id="MathJax-Span-16" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">n</span><span class="mo" id="MathJax-Span-17" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi><mo>=</mo><mo fence="false" stretchy="false">{</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mo>.</mo><mo>.</mo><mo>.</mo><mo>,</mo><mi>n</mi><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-1" type="math/tex">j \in J = \{1,2,...,n\}</script>: Index and set of jobs.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-2-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-18" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-19"><span class="mi" id="MathJax-Span-20" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-21" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-22" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-2" type="math/tex">k \in K</script>: Index and set of technicians.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2282;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-3-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-23" style="width: 5.301em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.408em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.41em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-24"><span class="mi" id="MathJax-Span-25" style="font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-26" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-27" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-28" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-29" style="font-family: MathJax_Main; padding-left: 0.301em;">⊂</span><span class="mi" id="MathJax-Span-30" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>K</mi><mo stretchy="false">(</mo><mi>j</mi><mo stretchy="false">)</mo><mo>⊂</mo><mi>K</mi></math></span></span><script id="MathJax-Element-3" type="math/tex">K(j) \subset K</script>: Subset of technicians qualified to perform job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-4-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-31" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-32"><span class="mi" id="MathJax-Span-33" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-34" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-35" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-4" type="math/tex">j \in J</script></p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;D&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;m&lt;/mi&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-5-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-36" style="width: 18.336em; display: inline-block;"><span style="display: inline-block; position: relative; width: 15.241em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1015.18em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-37"><span class="mi" id="MathJax-Span-38" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-39" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-40" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">D</span><span class="mo" id="MathJax-Span-41" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-42" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mi" id="MathJax-Span-43" style="font-family: MathJax_Math-italic;">n</span><span class="mo" id="MathJax-Span-44" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mn" id="MathJax-Span-45" style="font-family: MathJax_Main; padding-left: 0.241em;">1</span><span class="mo" id="MathJax-Span-46" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-47" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">n</span><span class="mo" id="MathJax-Span-48" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mn" id="MathJax-Span-49" style="font-family: MathJax_Main; padding-left: 0.241em;">2</span><span class="mo" id="MathJax-Span-50" style="font-family: MathJax_Main;">,</span><span class="mo" id="MathJax-Span-51" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-52" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-53" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-54" style="font-family: MathJax_Main; padding-left: 0.182em;">,</span><span class="mi" id="MathJax-Span-55" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">n</span><span class="mo" id="MathJax-Span-56" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-57" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">m</span><span class="mo" id="MathJax-Span-58" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>d</mi><mo>∈</mo><mi>D</mi><mo>=</mo><mo fence="false" stretchy="false">{</mo><mi>n</mi><mo>+</mo><mn>1</mn><mo>,</mo><mi>n</mi><mo>+</mo><mn>2</mn><mo>,</mo><mo>.</mo><mo>.</mo><mo>.</mo><mo>,</mo><mi>n</mi><mo>+</mo><mi>m</mi><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-5" type="math/tex">d \in D = \{n+1, n+2, ...,n+m \}</script>: Index and set of depots (service centers), where <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;m&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-6-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-59" style="width: 1.074em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.9em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-60"><span class="mi" id="MathJax-Span-61" style="font-family: MathJax_Math-italic;">m</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>m</mi></math></span></span><script id="MathJax-Element-6" type="math/tex">m</script> is the number of depots.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;l&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mo&gt;&amp;#x222A;&lt;/mo&gt;&lt;mi&gt;D&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;.&lt;/mo&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;m&lt;/mi&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-7-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-62" style="width: 20.182em; display: inline-block;"><span style="display: inline-block; position: relative; width: 16.789em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1016.73em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-63"><span class="mi" id="MathJax-Span-64" style="font-family: MathJax_Math-italic;">l</span><span class="mo" id="MathJax-Span-65" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-66" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">i</span><span class="mo" id="MathJax-Span-67" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-68" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-69" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-70" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">L</span><span class="mo" id="MathJax-Span-71" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mi" id="MathJax-Span-72" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-73" style="font-family: MathJax_Main; padding-left: 0.241em;">∪</span><span class="mi" id="MathJax-Span-74" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">D</span><span class="mo" id="MathJax-Span-75" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-76" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-77" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-78" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-79" style="font-family: MathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-80" style="font-family: MathJax_Main;">,</span><span class="mo" id="MathJax-Span-81" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-82" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-83" style="font-family: MathJax_Main; padding-left: 0.182em;">.</span><span class="mo" id="MathJax-Span-84" style="font-family: MathJax_Main; padding-left: 0.182em;">,</span><span class="mi" id="MathJax-Span-85" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">n</span><span class="mo" id="MathJax-Span-86" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-87" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">m</span><span class="mo" id="MathJax-Span-88" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>l</mi><mo>,</mo><mi>i</mi><mo>,</mo><mi>j</mi><mo>∈</mo><mi>L</mi><mo>=</mo><mi>J</mi><mo>∪</mo><mi>D</mi><mo>=</mo><mo fence="false" stretchy="false">{</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mo>.</mo><mo>.</mo><mo>.</mo><mo>,</mo><mi>n</mi><mo>+</mo><mi>m</mi><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-7" type="math/tex">l, i, j \in L = J \cup D = \{1,2,..., n+m \}</script>: index and set of locations to visit.</p>
<h3 id="Parameters">Parameters<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Parameters">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03B2;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-8-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-89" style="width: 6.372em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.301em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.24em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-90"><span class="msubsup" id="MathJax-Span-91"><span style="display: inline-block; position: relative; width: 1.551em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-92" style="font-family: MathJax_Math-italic;">β<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-93"><span class="mrow" id="MathJax-Span-94"><span class="mi" id="MathJax-Span-95" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-96" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-97" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-98" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-99" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-100" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-101" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-102" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-103" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>β</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>d</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-8" type="math/tex">\beta_{k,d} \in \{0,1\}</script>: This parameter is equal 1 if technician <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-9-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-104" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-105"><span class="mi" id="MathJax-Span-106" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-107" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-108" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-9" type="math/tex">k \in K</script> must depart from and return to depot 
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;D&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-10-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-109" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.62em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-110"><span class="mi" id="MathJax-Span-111" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-112" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-113" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">D</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>d</mi><mo>∈</mo><mi>D</mi></math></span></span><script id="MathJax-Element-10" type="math/tex">d \in D</script>; and 0 otherwise.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msup&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo mathvariant="double-struck"&gt;+&lt;/mo&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-11-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-114" style="width: 4.229em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.515em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1003.51em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-115"><span class="msubsup" id="MathJax-Span-116"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-117" style="font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-118"><span class="mrow" id="MathJax-Span-119"><span class="mi" id="MathJax-Span-120" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-121" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-122" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-123"><span class="msubsup" id="MathJax-Span-124"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-125" style="font-family: MathJax_AMS;">R</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-126"><span class="mrow" id="MathJax-Span-127"><span class="mo" id="MathJax-Span-128" style="font-size: 70.7%; font-family: MathJax_Main;">+</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>p</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><msup><mi mathvariant="double-struck">R</mi><mrow class="MJX-TeXAtom-ORD"><mo mathvariant="double-struck">+</mo></mrow></msup></mrow></math></span></span><script id="MathJax-Element-11" type="math/tex">p_{j} \in \mathbb{R^{+}}</script>; Processing time (duration) of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-12-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-129" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-130"><span class="mi" id="MathJax-Span-131" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-132" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-133" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-12" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C4;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msup&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo mathvariant="double-struck"&gt;+&lt;/mo&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-13-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-134" style="width: 4.646em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1003.87em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-135"><span class="msubsup" id="MathJax-Span-136"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-137" style="font-family: MathJax_Math-italic;">τ<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-138"><span class="mrow" id="MathJax-Span-139"><span class="mi" id="MathJax-Span-140" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-141" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-142" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-143" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-144" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-145"><span class="msubsup" id="MathJax-Span-146"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-147" style="font-family: MathJax_AMS;">R</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-148"><span class="mrow" id="MathJax-Span-149"><span class="mo" id="MathJax-Span-150" style="font-size: 70.7%; font-family: MathJax_Main;">+</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>τ</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><msup><mi mathvariant="double-struck">R</mi><mrow class="MJX-TeXAtom-ORD"><mo mathvariant="double-struck">+</mo></mrow></msup></mrow></math></span></span><script id="MathJax-Element-13" type="math/tex">\tau_{i,j} \in \mathbb{R^{+}}</script>: Travel time, in minutes, from location <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-14-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-151" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.26em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-152"><span class="mi" id="MathJax-Span-153" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-154" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-155" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">L</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><mi>L</mi></math></span></span><script id="MathJax-Element-14" type="math/tex">i \in L</script> to location <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-15-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-156" style="width: 2.86em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.384em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-157"><span class="mi" id="MathJax-Span-158" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-159" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-160" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">L</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>L</mi></math></span></span><script id="MathJax-Element-15" type="math/tex">j \in L</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;W&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-16-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-161" style="width: 4.051em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1003.34em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-162"><span class="msubsup" id="MathJax-Span-163"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1001.07em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-164" style="font-family: MathJax_Math-italic;">W<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.122em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.955em;"><span class="texatom" id="MathJax-Span-165"><span class="mrow" id="MathJax-Span-166"><span class="mi" id="MathJax-Span-167" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-168" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-169" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-170"><span class="mi" id="MathJax-Span-171" style="font-family: MathJax_AMS;">N</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>W</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow></math></span></span><script id="MathJax-Element-16" type="math/tex">W_{k} \in \mathbb{N}</script>: Workload limit of technician <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-17-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-172" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-173"><span class="mi" id="MathJax-Span-174" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-175" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-176" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-17" type="math/tex">k \in K</script> during the planning horizon. That is, the number of hours, in minutes, that the technician is available for the next work day.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;4&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-18-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-177" style="width: 7.979em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.61em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1006.55em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-178"><span class="msubsup" id="MathJax-Span-179"><span style="display: inline-block; position: relative; width: 0.955em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-180" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-181"><span class="mrow" id="MathJax-Span-182"><span class="mi" id="MathJax-Span-183" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-184" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-185" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-186" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-187" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-188" style="font-family: MathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-189" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-190" style="font-family: MathJax_Main; padding-left: 0.182em;">3</span><span class="mo" id="MathJax-Span-191" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-192" style="font-family: MathJax_Main; padding-left: 0.182em;">4</span><span class="mo" id="MathJax-Span-193" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>3</mn><mo>,</mo><mn>4</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-18" type="math/tex">\pi_{j} \in \{1,2,3,4\}</script>: Priority weight of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-19-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-194" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-195"><span class="mi" id="MathJax-Span-196" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-197" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-198" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-19" type="math/tex">j \in J</script>, a larger number means higher priority.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msup&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo mathvariant="double-struck"&gt;+&lt;/mo&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-20-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-199" style="width: 4.229em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.515em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1003.51em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-200"><span class="msubsup" id="MathJax-Span-201"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-202" style="font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.539em;"><span class="texatom" id="MathJax-Span-203"><span class="mrow" id="MathJax-Span-204"><span class="mi" id="MathJax-Span-205" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-206" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-207" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-208"><span class="msubsup" id="MathJax-Span-209"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-210" style="font-family: MathJax_AMS;">R</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-211"><span class="mrow" id="MathJax-Span-212"><span class="mo" id="MathJax-Span-213" style="font-size: 70.7%; font-family: MathJax_Main;">+</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>a</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><msup><mi mathvariant="double-struck">R</mi><mrow class="MJX-TeXAtom-ORD"><mo mathvariant="double-struck">+</mo></mrow></msup></mrow></math></span></span><script id="MathJax-Element-20" type="math/tex">a_{j} \in \mathbb{R^{+}} </script>: Earliest time to start the service at location of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-21-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-214" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-215"><span class="mi" id="MathJax-Span-216" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-217" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-218" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-21" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;b&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msup&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo mathvariant="double-struck"&gt;+&lt;/mo&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-22-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-219" style="width: 4.11em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.396em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1003.4em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-220"><span class="msubsup" id="MathJax-Span-221"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-222" style="font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-223"><span class="mrow" id="MathJax-Span-224"><span class="mi" id="MathJax-Span-225" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-226" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-227" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-228"><span class="msubsup" id="MathJax-Span-229"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-230" style="font-family: MathJax_AMS;">R</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-231"><span class="mrow" id="MathJax-Span-232"><span class="mo" id="MathJax-Span-233" style="font-size: 70.7%; font-family: MathJax_Main;">+</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>b</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><msup><mi mathvariant="double-struck">R</mi><mrow class="MJX-TeXAtom-ORD"><mo mathvariant="double-struck">+</mo></mrow></msup></mrow></math></span></span><script id="MathJax-Element-22" type="math/tex">b_{j} \in \mathbb{R^{+}} </script>: Latest time to start the service at location of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-23-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-234" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-235"><span class="mi" id="MathJax-Span-236" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-237" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-238" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-23" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msup&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo mathvariant="double-struck"&gt;+&lt;/mo&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-24-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-239" style="width: 4.884em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.05em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-240"><span class="mi" id="MathJax-Span-241" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="msubsup" id="MathJax-Span-242"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-243" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.539em;"><span class="texatom" id="MathJax-Span-244"><span class="mrow" id="MathJax-Span-245"><span class="mi" id="MathJax-Span-246" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-247" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-248" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-249"><span class="msubsup" id="MathJax-Span-250"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-251" style="font-family: MathJax_AMS;">R</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.717em;"><span class="texatom" id="MathJax-Span-252"><span class="mrow" id="MathJax-Span-253"><span class="mo" id="MathJax-Span-254" style="font-size: 70.7%; font-family: MathJax_Main;">+</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>d</mi><msub><mi>d</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><msup><mi mathvariant="double-struck">R</mi><mrow class="MJX-TeXAtom-ORD"><mo mathvariant="double-struck">+</mo></mrow></msup></mrow></math></span></span><script id="MathJax-Element-24" type="math/tex">dd_{j} \in \mathbb{R^{+}} </script>: Due date of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-25-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-255" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-256"><span class="mi" id="MathJax-Span-257" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-258" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-259" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-25" type="math/tex">j \in J</script>, i.e. the latest time to complete the service for job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-26-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-260" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-261"><span class="mi" id="MathJax-Span-262" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-263" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-264" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-26" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-27-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-265" style="width: 3.693em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.039em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1003.04em, 2.741em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-266"><span class="mi" id="MathJax-Span-267" style="font-family: MathJax_Math-italic;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-268" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="texatom" id="MathJax-Span-269" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-270"><span class="mi" id="MathJax-Span-271" style="font-family: MathJax_AMS;">N</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>M</mi><mo>∈</mo><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow></math></span></span><script id="MathJax-Element-27" type="math/tex">M \in \mathbb{N}</script>: This is a very large number. This number is determined as follows: the planning horizon is of 10 working hours (i.e. 600 min). We want the value of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-28-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-272" style="width: 1.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1001.01em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-273"><span class="mi" id="MathJax-Span-274" style="font-family: MathJax_Math-italic;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>M</mi></math></span></span><script id="MathJax-Element-28" type="math/tex">M</script> to be an order of magnitude larger than the planning horizon. Therefore, we set M = 6100.</p>
<h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-29-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-275" style="width: 6.313em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.241em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.18em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-276"><span class="msubsup" id="MathJax-Span-277"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-278" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-279"><span class="mrow" id="MathJax-Span-280"><span class="mi" id="MathJax-Span-281" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-282" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-283" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-284" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-285" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-286" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-287" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-288" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-289" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-29" type="math/tex">x_{j,k} \in \{0,1\}</script>: This variable is equal 1 if job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-30-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-290" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-291"><span class="mi" id="MathJax-Span-292" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-293" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-294" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-30" type="math/tex">j \in J</script> is assigned to technician <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-31-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-295" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-296"><span class="mi" id="MathJax-Span-297" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-298" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-299" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-31" type="math/tex">k \in K</script>, and 0 otherwise.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-32-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-300" style="width: 5.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.765em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.71em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-301"><span class="msubsup" id="MathJax-Span-302"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-303" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-304"><span class="mrow" id="MathJax-Span-305"><span class="mi" id="MathJax-Span-306" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-307" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-308" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-309" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-310" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-311" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-312" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>u</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-32" type="math/tex">u_{k} \in \{0,1\}</script>: This variable is equal 1 if technician <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-33-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-313" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-314"><span class="mi" id="MathJax-Span-315" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-316" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-317" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-33" type="math/tex">k \in K</script>  is used to perform a job, and 0 otherwise.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-34-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-318" style="width: 6.729em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.54em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-319"><span class="msubsup" id="MathJax-Span-320"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-321" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-322"><span class="mrow" id="MathJax-Span-323"><span class="mi" id="MathJax-Span-324" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-325" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-326" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-327" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-328" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-329" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-330" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-331" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-332" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-333" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-334" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-34" type="math/tex">y_{i,j,k} \in \{0,1\}</script>: This variable is equal 1 if technician <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-35-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-335" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1002.68em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-336"><span class="mi" id="MathJax-Span-337" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-338" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-339" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mi>K</mi></math></span></span><script id="MathJax-Element-35" type="math/tex">k \in K</script>  travels from location <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-36-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-340" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.26em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-341"><span class="mi" id="MathJax-Span-342" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-343" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-344" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">L</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>∈</mo><mi>L</mi></math></span></span><script id="MathJax-Element-36" type="math/tex">i \in L</script> to location 
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-37-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-345" style="width: 2.86em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.384em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-346"><span class="mi" id="MathJax-Span-347" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-348" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-349" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">L</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>L</mi></math></span></span><script id="MathJax-Element-37" type="math/tex">j \in L</script>; and 0 otherwise.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-38-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-350" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.56em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-351"><span class="msubsup" id="MathJax-Span-352"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-353" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-354"><span class="mrow" id="MathJax-Span-355"><span class="mi" id="MathJax-Span-356" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-357" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-358" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-38" type="math/tex">t_{j} \geq 0</script>: This variable determines the time to arrive or start the service at location <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-39-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-359" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-360"><span class="mi" id="MathJax-Span-361" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-362" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-363" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-39" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-40-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-364" style="width: 3.336em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.741em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.68em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-365"><span class="msubsup" id="MathJax-Span-366"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-367" style="font-family: MathJax_Math-italic;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-368"><span class="mrow" id="MathJax-Span-369"><span class="mi" id="MathJax-Span-370" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-371" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-372" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>z</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-40" type="math/tex">z_{j} \geq 0</script>: This variable determines the lateness of completing job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-41-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-373" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-374"><span class="mi" id="MathJax-Span-375" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-376" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-377" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-41" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;b&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-42-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-378" style="width: 6.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1005.12em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-379"><span class="mi" id="MathJax-Span-380" style="font-family: MathJax_Math-italic;">x</span><span class="msubsup" id="MathJax-Span-381"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-382" style="font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.539em;"><span class="texatom" id="MathJax-Span-383"><span class="mrow" id="MathJax-Span-384"><span class="mi" id="MathJax-Span-385" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-386" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-387" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">x</span><span class="msubsup" id="MathJax-Span-388"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-389" style="font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-390"><span class="mrow" id="MathJax-Span-391"><span class="mi" id="MathJax-Span-392" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-393" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-394" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>x</mi><msub><mi>a</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>,</mo><mi>x</mi><msub><mi>b</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-42" type="math/tex">xa_{j}, xb_{j} \geq 0</script>: Correction to earliest and latest time to start the service for job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-43-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-395" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-396"><span class="mi" id="MathJax-Span-397" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-398" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-399" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-43" type="math/tex">j \in J</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;g&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-44-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-400" style="width: 1.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.84em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-401"><span class="msubsup" id="MathJax-Span-402"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-403" style="font-family: MathJax_Math-italic;">g<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-404"><span class="mrow" id="MathJax-Span-405"><span class="mi" id="MathJax-Span-406" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>g</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-44" type="math/tex">g_{j}</script>: This variable is equal 1 if job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-45-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-407" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-408"><span class="mi" id="MathJax-Span-409" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-410" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-411" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-45" type="math/tex">j \in J</script> cannot be filled, and 0 otherwise.</p>
<h3 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Objective-Function">¶</a></h3><ul>
<li><strong>Minimize lateness:</strong> The Objective function is to minimize the total weighted lateness of all the jobs. </li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-0"&gt;&lt;mtext&gt;(0)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mn&gt;0.01&lt;/mn&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;b&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;g&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-46-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-412" style="width: 100%; display: inline-block; min-width: 28.574em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 28.574em;"><span style="position: absolute; clip: rect(2.443em, 1024.41em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-413"><span class="mtable" id="MathJax-Span-414" style="min-width: 28.574em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 28.574em;"><span style="display: inline-block; position: absolute; width: 24.408em; height: 0px; clip: rect(-1.545em, 1024.41em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -12.199em;"><span style="position: absolute; clip: rect(2.443em, 1024.41em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 24.408em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1024.41em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -12.199em;"><span class="mtd" id="MathJax-Span-418"><span class="mrow" id="MathJax-Span-419"><span class="munderover" id="MathJax-Span-420"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-421" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-422"><span class="mrow" id="MathJax-Span-423"><span class="mi" id="MathJax-Span-424" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-425" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-426" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-427" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 0.955em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-428" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-429"><span class="mrow" id="MathJax-Span-430"><span class="mi" id="MathJax-Span-431" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-432" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-433" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-434" style="font-family: MathJax_Math-italic;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-435"><span class="mrow" id="MathJax-Span-436"><span class="mi" id="MathJax-Span-437" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-438" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-439" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-440" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-441"><span class="mrow" id="MathJax-Span-442"><span class="mi" id="MathJax-Span-443" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-444" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-445" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mn" id="MathJax-Span-446" style="font-family: MathJax_Main; padding-left: 0.182em;">0.01</span><span class="mo" id="MathJax-Span-447" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-448" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.955em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-449" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-450"><span class="mrow" id="MathJax-Span-451"><span class="mi" id="MathJax-Span-452" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-453" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mi" id="MathJax-Span-454" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-455" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-456" style="font-family: MathJax_Math-italic;">x</span><span class="msubsup" id="MathJax-Span-457"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-458" style="font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.539em;"><span class="texatom" id="MathJax-Span-459"><span class="mrow" id="MathJax-Span-460"><span class="mi" id="MathJax-Span-461" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-462" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-463" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">x</span><span class="msubsup" id="MathJax-Span-464"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-465" style="font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-466"><span class="mrow" id="MathJax-Span-467"><span class="mi" id="MathJax-Span-468" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-469" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-470" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-471" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-472" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-473"><span class="mrow" id="MathJax-Span-474"><span class="mi" id="MathJax-Span-475" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-476" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-477" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-478" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 0.955em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-479" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-480"><span class="mrow" id="MathJax-Span-481"><span class="mi" id="MathJax-Span-482" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-483" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mi" id="MathJax-Span-484" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-485" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-486" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-487" style="font-family: MathJax_Math-italic;">g<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-488"><span class="mrow" id="MathJax-Span-489"><span class="mi" id="MathJax-Span-490" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-0"><span class="mrow" id="MathJax-Span-416"><span class="mtext" id="MathJax-Span-417" style="font-family: MathJax_Main;">(0)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-0"><mtext>(0)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>z</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><mn>0.01</mn><mo>⋅</mo><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>⋅</mo><mi>M</mi><mo stretchy="false">(</mo><mi>x</mi><msub><mi>a</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>+</mo><mi>x</mi><msub><mi>b</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo stretchy="false">)</mo><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>⋅</mo><mi>M</mi><mo>⋅</mo><msub><mi>g</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-46" type="math/tex; mode=display">\begin{equation}
\sum_{j \in J} \pi_{j} \cdot z_{j} + \sum_{j \in J} 0.01 \cdot \pi_{j} \cdot M(xa_{j} + xb_{j}) + 
\sum_{j \in J} \pi_{j} \cdot M \cdot g_{j}
\tag{0}
\end{equation}</script><p><strong>Note</strong>: We want to treat the constraints of filling the jobs demand and the constraints about starting the jobs within a time window as soft constraints. Soft constraints can be violated, but the violation will incur in a huge penalty. To consider that each job has different priority, we use the priority penalty for the calculation of the penalty associated with a violation of soft constraints. We assume that not filling jobs demand greatly deteriorate customer satisfaction, consequently should incur the largest penalties associated with the soft constraints. Recall that the value of the paramter <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-47-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-491" style="width: 1.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1001.01em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-492"><span class="mi" id="MathJax-Span-493" style="font-family: MathJax_Math-italic;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>M</mi></math></span></span><script id="MathJax-Element-47" type="math/tex">M</script> is a large number, then the penalty associated with not filling a job is determined as follows: <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-48-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-494" style="width: 3.336em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.741em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.74em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-495"><span class="msubsup" id="MathJax-Span-496"><span style="display: inline-block; position: relative; width: 0.955em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-497" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-498"><span class="mrow" id="MathJax-Span-499"><span class="mi" id="MathJax-Span-500" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-501" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mi" id="MathJax-Span-502" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>⋅</mo><mi>M</mi></math></span></span><script id="MathJax-Element-48" type="math/tex">\pi_{j} \cdot M</script>. The time windows constraints violation also deteriorates customer satisfaction but to a lesser degree than the penalty associated with violating the time windows constraints. The time window constraints are determined as follows: <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mn&gt;0.01&lt;/mn&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-49-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-503" style="width: 6.372em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.301em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1005.3em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-504"><span class="mn" id="MathJax-Span-505" style="font-family: MathJax_Main;">0.01</span><span class="mo" id="MathJax-Span-506" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-507" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.955em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-508" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-509"><span class="mrow" id="MathJax-Span-510"><span class="mi" id="MathJax-Span-511" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-512" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mi" id="MathJax-Span-513" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mn>0.01</mn><mo>⋅</mo><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>⋅</mo><mi>M</mi></math></span></span><script id="MathJax-Element-49" type="math/tex">0.01 \cdot \pi_{j} \cdot M</script>.</p>
<h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Constraints">¶</a></h3><ul>
<li><strong>Assign qualified technicians:</strong> For each job, we  assign one technician who is qualified for the job, or a gap is declared. </li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-1"&gt;&lt;mtext&gt;(1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;g&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-50-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-514" style="width: 100%; display: inline-block; min-width: 16.015em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 16.015em;"><span style="position: absolute; clip: rect(2.384em, 1011.85em, 5.122em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-515"><span class="mtable" id="MathJax-Span-516" style="min-width: 16.015em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 16.015em;"><span style="display: inline-block; position: absolute; width: 11.848em; height: 0px; clip: rect(-1.604em, 1011.85em, 1.134em, -999.997em); top: 0em; left: 50%; margin-left: -5.949em;"><span style="position: absolute; clip: rect(2.384em, 1011.85em, 5.122em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 11.848em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1011.85em, 5.598em, -999.997em); top: -4.461em; left: 50%; margin-left: -5.949em;"><span class="mtd" id="MathJax-Span-520"><span class="mrow" id="MathJax-Span-521"><span class="munderover" id="MathJax-Span-522"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.42em;"><span class="mo" id="MathJax-Span-523" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1002.26em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-524"><span class="mrow" id="MathJax-Span-525"><span class="mi" id="MathJax-Span-526" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-527" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-528" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-529" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-530" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-531" style="font-size: 70.7%; font-family: MathJax_Main;">)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-532" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-533" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-534"><span class="mrow" id="MathJax-Span-535"><span class="mi" id="MathJax-Span-536" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-537" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-538" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-539" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-540" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-541" style="font-family: MathJax_Math-italic;">g<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-542"><span class="mrow" id="MathJax-Span-543"><span class="mi" id="MathJax-Span-544" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-545" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-546" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-547" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-548" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-549" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-550" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-551" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.426em, 1001.19em, -0.057em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-1"><span class="mrow" id="MathJax-Span-518"><span class="mtext" id="MathJax-Span-519" style="font-family: MathJax_Main;">(1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.211em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-1"><mtext>(1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi><mo stretchy="false">(</mo><mi>j</mi><mo stretchy="false">)</mo></mrow></munder><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>+</mo><msub><mi>g</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>=</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><mi>J</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-50" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K(j)} x_{j,k} + g_{j} = 1 \quad \forall j \in J
\tag{1}
\end{equation}</script><p><strong>Note</strong>: The penalty of the gap variable <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;g&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-51-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-552" style="width: 1.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.84em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-553"><span class="msubsup" id="MathJax-Span-554"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-555" style="font-family: MathJax_Math-italic;">g<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-556"><span class="mrow" id="MathJax-Span-557"><span class="mi" id="MathJax-Span-558" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>g</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-51" type="math/tex">g_{j}</script> is (<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mn&gt;0.1&lt;/mn&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-52-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-559" style="width: 5.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.765em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1004.76em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-560"><span class="mn" id="MathJax-Span-561" style="font-family: MathJax_Main;">0.1</span><span class="mo" id="MathJax-Span-562" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-563" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.955em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-564" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-565"><span class="mrow" id="MathJax-Span-566"><span class="mi" id="MathJax-Span-567" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-568" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mi" id="MathJax-Span-569" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mn>0.1</mn><mo>⋅</mo><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>⋅</mo><mi>M</mi></math></span></span><script id="MathJax-Element-52" type="math/tex">0.1 \cdot \pi_{j} \cdot M</script>) which is a large number to discourage 
not being able to satisfy demand.</p>
<ul>
<li><strong>Only one technician:</strong> For each job, we only allow one technician to be assigned.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-2"&gt;&lt;mtext&gt;(2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-53-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-570" style="width: 100%; display: inline-block; min-width: 13.039em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 13.039em;"><span style="position: absolute; clip: rect(2.443em, 1008.93em, 5.003em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-571"><span class="mtable" id="MathJax-Span-572" style="min-width: 13.039em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 13.039em;"><span style="display: inline-block; position: absolute; width: 8.932em; height: 0px; clip: rect(-1.545em, 1008.93em, 1.015em, -999.997em); top: 0em; left: 50%; margin-left: -4.461em;"><span style="position: absolute; clip: rect(2.443em, 1008.93em, 5.003em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 8.932em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1008.93em, 5.42em, -999.997em); top: -4.402em; left: 50%; margin-left: -4.461em;"><span class="mtd" id="MathJax-Span-576"><span class="mrow" id="MathJax-Span-577"><span class="munderover" id="MathJax-Span-578"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-579" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-580"><span class="mrow" id="MathJax-Span-581"><span class="mi" id="MathJax-Span-582" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-583" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-584" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-585" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-586" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-587"><span class="mrow" id="MathJax-Span-588"><span class="mi" id="MathJax-Span-589" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-590" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-591" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-592" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="mn" id="MathJax-Span-593" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-594" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-595" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-596" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-597" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-598" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.307em, 1001.19em, 0.063em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.402em; right: 0em;"><span class="mtd" id="mjx-eqn-2"><span class="mrow" id="MathJax-Span-574"><span class="mtext" id="MathJax-Span-575" style="font-family: MathJax_Main;">(2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.068em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-2"><mtext>(2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>≤</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><mi>J</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-53" type="math/tex; mode=display">\begin{equation}
\sum_{k \in K} x_{j,k} \leq 1 \quad \forall j \in J
\tag{2}
\end{equation}</script><ul>
<li><strong>Technician capacity:</strong> For each technician, we ensure that the available capacity of the technician is not exceeded. </li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-3"&gt;&lt;mtext&gt;(3)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C4;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;W&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-54-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-599" style="width: 100%; display: inline-block; min-width: 26.074em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 26.074em;"><span style="position: absolute; clip: rect(2.443em, 1021.91em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-600"><span class="mtable" id="MathJax-Span-601" style="min-width: 26.074em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 26.074em;"><span style="display: inline-block; position: absolute; width: 21.908em; height: 0px; clip: rect(-1.545em, 1021.91em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -10.949em;"><span style="position: absolute; clip: rect(2.443em, 1021.91em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 21.908em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1021.91em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -10.949em;"><span class="mtd" id="MathJax-Span-605"><span class="mrow" id="MathJax-Span-606"><span class="munderover" id="MathJax-Span-607"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-608" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-609"><span class="mrow" id="MathJax-Span-610"><span class="mi" id="MathJax-Span-611" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-612" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-613" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-614" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-615" style="font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-616"><span class="mrow" id="MathJax-Span-617"><span class="mi" id="MathJax-Span-618" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-619" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-620" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-621" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-622"><span class="mrow" id="MathJax-Span-623"><span class="mi" id="MathJax-Span-624" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-625" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-626" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-627" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-628" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-629" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.289em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-630"><span class="mrow" id="MathJax-Span-631"><span class="mi" id="MathJax-Span-632" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-633" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-634" style="font-size: 70.7%; font-family: MathJax_Math-italic;">L</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="munderover" id="MathJax-Span-635" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-636" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.25em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-637"><span class="mrow" id="MathJax-Span-638"><span class="mi" id="MathJax-Span-639" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-640" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-641" style="font-size: 70.7%; font-family: MathJax_Math-italic;">L</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-642" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-643" style="font-family: MathJax_Math-italic;">τ<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-644"><span class="mrow" id="MathJax-Span-645"><span class="mi" id="MathJax-Span-646" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-647" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-648" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-649" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-650" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-651" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-652"><span class="mrow" id="MathJax-Span-653"><span class="mi" id="MathJax-Span-654" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-655" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-656" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-657" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-658" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-659" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-660" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1001.07em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-661" style="font-family: MathJax_Math-italic;">W<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.122em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.955em;"><span class="texatom" id="MathJax-Span-662"><span class="mrow" id="MathJax-Span-663"><span class="mi" id="MathJax-Span-664" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-665" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-666" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-667" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-668"><span class="mrow" id="MathJax-Span-669"><span class="mi" id="MathJax-Span-670" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-671" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-672" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-673" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-674" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-675" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-3"><span class="mrow" id="MathJax-Span-603"><span class="mtext" id="MathJax-Span-604" style="font-family: MathJax_Main;">(3)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-3"><mtext>(3)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>p</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>L</mi></mrow></munder><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>L</mi></mrow></munder><msub><mi>τ</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>⋅</mo><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>≤</mo><msub><mi>W</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mi>u</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-54" type="math/tex; mode=display">\begin{equation}
\sum_{j \in J}  p_{j} \cdot x_{j,k} + \sum_{i \in L} \sum_{j \in L} \tau_{i,j} \cdot y_{i,j,k} \leq W_{k} \cdot u_{k} \quad \forall k \in K
\tag{3}
\end{equation}</script><ul>
<li><strong>Technician tour:</strong> For each technician and job, we ensure that if the technician is assigned to the job, then the technician must travel to another location (to form a tour). </li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-4"&gt;&lt;mtext&gt;(4)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-55-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-676" style="width: 100%; display: inline-block; min-width: 17.443em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 17.443em;"><span style="position: absolute; clip: rect(2.443em, 1013.28em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-677"><span class="mtable" id="MathJax-Span-678" style="min-width: 17.443em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 17.443em;"><span style="display: inline-block; position: absolute; width: 13.277em; height: 0px; clip: rect(-1.545em, 1013.28em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -6.604em;"><span style="position: absolute; clip: rect(2.443em, 1013.28em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 13.277em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1013.28em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -6.604em;"><span class="mtd" id="MathJax-Span-682"><span class="mrow" id="MathJax-Span-683"><span class="munderover" id="MathJax-Span-684"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-685" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.25em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-686"><span class="mrow" id="MathJax-Span-687"><span class="mi" id="MathJax-Span-688" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-689" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-690" style="font-size: 70.7%; font-family: MathJax_Math-italic;">L</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-691" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-692" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-693"><span class="mrow" id="MathJax-Span-694"><span class="mi" id="MathJax-Span-695" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-696" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-697" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-698" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-699" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-700" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-701" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-702" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-703"><span class="mrow" id="MathJax-Span-704"><span class="mi" id="MathJax-Span-705" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-706" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-707" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-708" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-709" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-710" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-711" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-712" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-713" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-714" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-715" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-716" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-4"><span class="mrow" id="MathJax-Span-680"><span class="mtext" id="MathJax-Span-681" style="font-family: MathJax_Main;">(4)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-4"><mtext>(4)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>L</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>=</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>k</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>∈</mo><mi>J</mi><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-55" type="math/tex; mode=display">\begin{equation}
\sum_{j \in L}  y_{i,j,k} = x_{i,k} \quad \forall i \in J, k \in K
\tag{4}
\end{equation}</script><ul>
<li>For each technician and job, we ensure that if a technician is assigned to the job, then the technician must travel from another location to the location of the job (to form a tour). </li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-5"&gt;&lt;mtext&gt;(5)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-56-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-717" style="width: 100%; display: inline-block; min-width: 17.563em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 17.563em;"><span style="position: absolute; clip: rect(2.503em, 1013.4em, 5.003em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-718"><span class="mtable" id="MathJax-Span-719" style="min-width: 17.563em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 17.563em;"><span style="display: inline-block; position: absolute; width: 13.396em; height: 0px; clip: rect(-1.485em, 1013.4em, 1.015em, -999.997em); top: 0em; left: 50%; margin-left: -6.664em;"><span style="position: absolute; clip: rect(2.503em, 1013.4em, 5.003em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 13.396em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1013.4em, 5.42em, -999.997em); top: -4.402em; left: 50%; margin-left: -6.664em;"><span class="mtd" id="MathJax-Span-723"><span class="mrow" id="MathJax-Span-724"><span class="munderover" id="MathJax-Span-725"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-726" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.289em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-727"><span class="mrow" id="MathJax-Span-728"><span class="mi" id="MathJax-Span-729" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-730" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-731" style="font-size: 70.7%; font-family: MathJax_Math-italic;">L</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-732" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-733" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-734"><span class="mrow" id="MathJax-Span-735"><span class="mi" id="MathJax-Span-736" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-737" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-738" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-739" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-740" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-741" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-742" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-743" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-744"><span class="mrow" id="MathJax-Span-745"><span class="mi" id="MathJax-Span-746" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-747" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-748" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-749" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-750" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-751" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-752" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-753" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-754" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-755" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-756" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-757" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.307em, 1001.19em, 0.063em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.402em; right: 0em;"><span class="mtd" id="mjx-eqn-5"><span class="mrow" id="MathJax-Span-721"><span class="mtext" id="MathJax-Span-722" style="font-family: MathJax_Main;">(5)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.068em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-5"><mtext>(5)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>L</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>=</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><mi>J</mi><mo>,</mo><mi>k</mi><mo>∈</mo><mi>K</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-56" type="math/tex; mode=display">\begin{equation}
\sum_{i \in L}  y_{i,j,k} = x_{j,k} \quad \forall j \in J, k \in K
\tag{5}
\end{equation}</script><ul>
<li><strong>Same depot:</strong> For each technician and depot, we ensure that a technician, if assigned to any job, must depart from and return to the service center (depot) where the technician is based. </li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-6"&gt;&lt;mtext&gt;(6)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03B2;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;D&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-57-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-758" style="width: 100%; display: inline-block; min-width: 19.824em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 19.824em;"><span style="position: absolute; clip: rect(2.443em, 1015.66em, 5.063em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-759"><span class="mtable" id="MathJax-Span-760" style="min-width: 19.824em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 19.824em;"><span style="display: inline-block; position: absolute; width: 15.658em; height: 0px; clip: rect(-1.545em, 1015.66em, 1.074em, -999.997em); top: 0em; left: 50%; margin-left: -7.854em;"><span style="position: absolute; clip: rect(2.443em, 1015.66em, 5.063em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 15.658em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1015.66em, 5.539em, -999.997em); top: -4.461em; left: 50%; margin-left: -7.854em;"><span class="mtd" id="MathJax-Span-764"><span class="mrow" id="MathJax-Span-765"><span class="munderover" id="MathJax-Span-766"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-767" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.408em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-768"><span class="mrow" id="MathJax-Span-769"><span class="mi" id="MathJax-Span-770" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-771" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-772" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-773" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-774" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-775"><span class="mrow" id="MathJax-Span-776"><span class="mi" id="MathJax-Span-777" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-778" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-779" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-780" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-781" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-782" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-783" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.551em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-784" style="font-family: MathJax_Math-italic;">β<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-785"><span class="mrow" id="MathJax-Span-786"><span class="mi" id="MathJax-Span-787" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-788" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-789" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-790" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-791" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-792" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-793"><span class="mrow" id="MathJax-Span-794"><span class="mi" id="MathJax-Span-795" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-796" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-797" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-798" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-799" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-800" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-801" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-802" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-803" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-804" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">D</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.366em, 1001.19em, 0.003em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.461em; right: 0em;"><span class="mtd" id="mjx-eqn-6"><span class="mrow" id="MathJax-Span-762"><span class="mtext" id="MathJax-Span-763" style="font-family: MathJax_Main;">(6)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.139em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-6"><mtext>(6)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>d</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>=</mo><msub><mi>β</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>d</mi></mrow></msub><mo>⋅</mo><msub><mi>u</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>k</mi><mo>∈</mo><mi>K</mi><mo>,</mo><mi>d</mi><mo>∈</mo><mi>D</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-57" type="math/tex; mode=display">\begin{equation}
\sum_{j \in J}  y_{d,j,k} = \beta_{k,d} \cdot u_{k} \quad \forall  k \in K, d \in D
\tag{6}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-7"&gt;&lt;mtext&gt;(7)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03B2;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;D&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-58-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-805" style="width: 100%; display: inline-block; min-width: 19.765em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 19.765em;"><span style="position: absolute; clip: rect(2.503em, 1015.6em, 5.003em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-806"><span class="mtable" id="MathJax-Span-807" style="min-width: 19.765em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 19.765em;"><span style="display: inline-block; position: absolute; width: 15.598em; height: 0px; clip: rect(-1.485em, 1015.6em, 1.015em, -999.997em); top: 0em; left: 50%; margin-left: -7.795em;"><span style="position: absolute; clip: rect(2.503em, 1015.6em, 5.003em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 15.598em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1015.6em, 5.42em, -999.997em); top: -4.402em; left: 50%; margin-left: -7.795em;"><span class="mtd" id="MathJax-Span-811"><span class="mrow" id="MathJax-Span-812"><span class="munderover" id="MathJax-Span-813"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-814" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.19em, 4.289em, -999.997em); top: -2.914em; left: 0.122em;"><span class="texatom" id="MathJax-Span-815"><span class="mrow" id="MathJax-Span-816"><span class="mi" id="MathJax-Span-817" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-818" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-819" style="font-size: 70.7%; font-family: MathJax_Math-italic;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-820" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.967em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-821" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-822"><span class="mrow" id="MathJax-Span-823"><span class="mi" id="MathJax-Span-824" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-825" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-826" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-827" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-828" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-829" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-830" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.551em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-831" style="font-family: MathJax_Math-italic;">β<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-832"><span class="mrow" id="MathJax-Span-833"><span class="mi" id="MathJax-Span-834" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-835" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-836" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-837" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-838" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-839" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-840"><span class="mrow" id="MathJax-Span-841"><span class="mi" id="MathJax-Span-842" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-843" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-844" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-845" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-846" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-847" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-848" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-849" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-850" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-851" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">D</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.307em, 1001.19em, 0.063em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.402em; right: 0em;"><span class="mtd" id="mjx-eqn-7"><span class="mrow" id="MathJax-Span-809"><span class="mtext" id="MathJax-Span-810" style="font-family: MathJax_Main;">(7)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.068em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-7"><mtext>(7)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>J</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>d</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>=</mo><msub><mi>β</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>d</mi></mrow></msub><mo>⋅</mo><msub><mi>u</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>k</mi><mo>∈</mo><mi>K</mi><mo>,</mo><mi>d</mi><mo>∈</mo><mi>D</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-58" type="math/tex; mode=display">\begin{equation}
\sum_{i \in J}  y_{i,d,k} = \beta_{k,d} \cdot u_{k} \quad \forall  k \in K, d \in D
\tag{7}
\end{equation}</script><ul>
<li><strong>Temporal relationship:</strong> For each location and job, we ensure the temporal relationship between two consecutive jobs served by the same technician. That is, if a technician <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-59-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-852" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-853"><span class="mi" id="MathJax-Span-854" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-59" type="math/tex">k</script> travels from job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-60-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-855" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-856"><span class="mi" id="MathJax-Span-857" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-60" type="math/tex">i</script> to job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-61-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-858" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-859"><span class="mi" id="MathJax-Span-860" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-61" type="math/tex">j</script>, then the start of the service time at job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-62-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-861" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-862"><span class="mi" id="MathJax-Span-863" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-62" type="math/tex">j</script> must be no less than the completion time of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-63-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-864" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-865"><span class="mi" id="MathJax-Span-866" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-63" type="math/tex">i</script> plus the travel time from job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-64-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-867" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-868"><span class="mi" id="MathJax-Span-869" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-64" type="math/tex">i</script> to job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-65-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-870" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-871"><span class="mi" id="MathJax-Span-872" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-65" type="math/tex">j</script>. </li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-8"&gt;&lt;mtext&gt;(8)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C4;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;L&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-66-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-873" style="width: 100%; display: inline-block; min-width: 27.265em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 27.265em;"><span style="position: absolute; clip: rect(2.443em, 1023.1em, 5.003em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-874"><span class="mtable" id="MathJax-Span-875" style="min-width: 27.265em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 27.265em;"><span style="display: inline-block; position: absolute; width: 23.098em; height: 0px; clip: rect(-1.545em, 1023.1em, 1.015em, -999.997em); top: 0em; left: 50%; margin-left: -11.545em;"><span style="position: absolute; clip: rect(2.443em, 1023.1em, 5.003em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 23.098em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1023.1em, 5.42em, -999.997em); top: -4.402em; left: 50%; margin-left: -11.545em;"><span class="mtd" id="MathJax-Span-879"><span class="mrow" id="MathJax-Span-880"><span class="msubsup" id="MathJax-Span-881"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-882" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-883"><span class="mrow" id="MathJax-Span-884"><span class="mi" id="MathJax-Span-885" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-886" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-887" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 0.658em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-888" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-889"><span class="mrow" id="MathJax-Span-890"><span class="mi" id="MathJax-Span-891" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-892" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-893" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-894" style="font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-895"><span class="mrow" id="MathJax-Span-896"><span class="mi" id="MathJax-Span-897" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-898" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-899" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-900" style="font-family: MathJax_Math-italic;">τ<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-901"><span class="mrow" id="MathJax-Span-902"><span class="mi" id="MathJax-Span-903" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-904" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-905" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-906" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mi" id="MathJax-Span-907" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-908" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mo" id="MathJax-Span-909" style="font-family: MathJax_Main; padding-left: 0.241em;">(</span><span class="mn" id="MathJax-Span-910" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-911" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="munderover" id="MathJax-Span-912" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.491em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.003em;"><span class="mo" id="MathJax-Span-913" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1001.49em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-914"><span class="mrow" id="MathJax-Span-915"><span class="mi" id="MathJax-Span-916" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-917" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-918" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-919" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-920" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-921"><span class="mrow" id="MathJax-Span-922"><span class="mi" id="MathJax-Span-923" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-924" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-925" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-926" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-927" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-928" style="font-family: MathJax_Main;">)</span><span class="mspace" id="MathJax-Span-929" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-930" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-931" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-932" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-933" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">L</span><span class="mo" id="MathJax-Span-934" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-935" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-936" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-937" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.307em, 1001.19em, 0.063em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.402em; right: 0em;"><span class="mtd" id="mjx-eqn-8"><span class="mrow" id="MathJax-Span-877"><span class="mtext" id="MathJax-Span-878" style="font-family: MathJax_Main;">(8)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.068em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-8"><mtext>(8)</mtext></mtd><mtd><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi></mrow></msub><mo>+</mo><msub><mi>p</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi></mrow></msub><mo>+</mo><msub><mi>τ</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>−</mo><mi>M</mi><mo>⋅</mo><mo stretchy="false">(</mo><mn>1</mn><mo>−</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo stretchy="false">)</mo><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>i</mi><mo>∈</mo><mi>L</mi><mo>,</mo><mi>j</mi><mo>∈</mo><mi>J</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-66" type="math/tex; mode=display">\begin{equation}
t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) \quad \forall i \in L, j \in J
\tag{8}
\end{equation}</script><p><strong>Note</strong>: Observe that if the technician <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-67-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-938" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-939"><span class="mi" id="MathJax-Span-940" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-67" type="math/tex">k</script> travels from the location of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-68-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-941" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-942"><span class="mi" id="MathJax-Span-943" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-68" type="math/tex">i</script> to the location of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-69-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-944" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-945"><span class="mi" id="MathJax-Span-946" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-69" type="math/tex">j</script>, then
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-70-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-947" style="width: 7.86em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.551em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1006.49em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-948"><span class="munderover" id="MathJax-Span-949"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1001.01em, 4.408em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-950" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.688em; left: 1.074em;"><span class="texatom" id="MathJax-Span-951"><span class="mrow" id="MathJax-Span-952"><span class="mi" id="MathJax-Span-953" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-954" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-955" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-956" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-957" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-958"><span class="mrow" id="MathJax-Span-959"><span class="mi" id="MathJax-Span-960" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-961" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-962" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-963" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-964" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-965" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-966" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>=</mo><mn>1</mn></math></span></span><script id="MathJax-Element-70" type="math/tex">\sum_{k \in K}  y_{i,j,k} = 1</script>. Therefore, <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-71-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-967" style="width: 13.039em; display: inline-block;"><span style="display: inline-block; position: relative; width: 10.836em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1010.78em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-968"><span class="mi" id="MathJax-Span-969" style="font-family: MathJax_Math-italic;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-970" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mo" id="MathJax-Span-971" style="font-family: MathJax_Main; padding-left: 0.241em;">(</span><span class="mn" id="MathJax-Span-972" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-973" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="munderover" id="MathJax-Span-974" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1001.01em, 4.408em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-975" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.688em; left: 1.074em;"><span class="texatom" id="MathJax-Span-976"><span class="mrow" id="MathJax-Span-977"><span class="mi" id="MathJax-Span-978" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-979" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-980" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-981" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-982" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-983"><span class="mrow" id="MathJax-Span-984"><span class="mi" id="MathJax-Span-985" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-986" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-987" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-988" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-989" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-990" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-991" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-992" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>M</mi><mo>⋅</mo><mo stretchy="false">(</mo><mn>1</mn><mo>−</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo stretchy="false">)</mo><mo>=</mo><mn>0</mn></math></span></span><script id="MathJax-Element-71" type="math/tex">M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = 0</script>, and the constraint 
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C4;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-72-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-993" style="width: 8.872em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.384em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1007.38em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-994"><span class="msubsup" id="MathJax-Span-995"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-996" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-997"><span class="mrow" id="MathJax-Span-998"><span class="mi" id="MathJax-Span-999" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1000" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-1001" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 0.658em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1002" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-1003"><span class="mrow" id="MathJax-Span-1004"><span class="mi" id="MathJax-Span-1005" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1006" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-1007" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1008" style="font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1009"><span class="mrow" id="MathJax-Span-1010"><span class="mi" id="MathJax-Span-1011" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1012" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-1013" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1014" style="font-family: MathJax_Math-italic;">τ<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1015"><span class="mrow" id="MathJax-Span-1016"><span class="mi" id="MathJax-Span-1017" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1018" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1019" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi></mrow></msub><mo>+</mo><msub><mi>p</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi></mrow></msub><mo>+</mo><msub><mi>τ</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-72" type="math/tex">t_{j} \geq t_{i} + p_{i} + \tau_{i,j}</script> would be properly enforced. Now consider the case where the technician <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-73-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1020" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1021"><span class="mi" id="MathJax-Span-1022" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-73" type="math/tex">k</script> does not travel  from the location of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-74-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1023" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1024"><span class="mi" id="MathJax-Span-1025" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-74" type="math/tex">i</script> to the location of job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-75-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1026" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1027"><span class="mi" id="MathJax-Span-1028" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-75" type="math/tex">j</script>. Hence, <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-76-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1029" style="width: 7.86em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.551em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1006.49em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1030"><span class="munderover" id="MathJax-Span-1031"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1001.01em, 4.408em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1032" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.688em; left: 1.074em;"><span class="texatom" id="MathJax-Span-1033"><span class="mrow" id="MathJax-Span-1034"><span class="mi" id="MathJax-Span-1035" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1036" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1037" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1038" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1039" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1040"><span class="mrow" id="MathJax-Span-1041"><span class="mi" id="MathJax-Span-1042" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1043" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1044" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1045" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1046" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1047" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-1048" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>=</mo><mn>0</mn></math></span></span><script id="MathJax-Element-76" type="math/tex">\sum_{k \in K}  y_{i,j,k} = 0</script> and 
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;K&lt;/mi&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-77-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1049" style="width: 13.693em; display: inline-block;"><span style="display: inline-block; position: relative; width: 11.372em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1011.37em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1050"><span class="mi" id="MathJax-Span-1051" style="font-family: MathJax_Math-italic;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1052" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mo" id="MathJax-Span-1053" style="font-family: MathJax_Main; padding-left: 0.241em;">(</span><span class="mn" id="MathJax-Span-1054" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-1055" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="munderover" id="MathJax-Span-1056" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1001.01em, 4.408em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1057" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.688em; left: 1.074em;"><span class="texatom" id="MathJax-Span-1058"><span class="mrow" id="MathJax-Span-1059"><span class="mi" id="MathJax-Span-1060" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-1061" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1062" style="font-size: 70.7%; font-family: MathJax_Math-italic;">K<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-1063" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 1.848em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1064" style="font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1065"><span class="mrow" id="MathJax-Span-1066"><span class="mi" id="MathJax-Span-1067" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1068" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1069" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1070" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1071" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1072" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-1073" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mi" id="MathJax-Span-1074" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>M</mi><mo>⋅</mo><mo stretchy="false">(</mo><mn>1</mn><mo>−</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>∈</mo><mi>K</mi></mrow></munder><msub><mi>y</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi><mo>,</mo><mi>k</mi></mrow></msub><mo stretchy="false">)</mo><mo>=</mo><mi>M</mi></math></span></span><script id="MathJax-Element-77" type="math/tex">M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = M</script>. In this case, this constraint becomes 
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C4;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-78-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1075" style="width: 11.61em; display: inline-block;"><span style="display: inline-block; position: relative; width: 9.646em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1009.65em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1076"><span class="msubsup" id="MathJax-Span-1077"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1078" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-1079"><span class="mrow" id="MathJax-Span-1080"><span class="mi" id="MathJax-Span-1081" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1082" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-1083" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 0.658em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1084" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-1085"><span class="mrow" id="MathJax-Span-1086"><span class="mi" id="MathJax-Span-1087" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1088" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-1089" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1090" style="font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1091"><span class="mrow" id="MathJax-Span-1092"><span class="mi" id="MathJax-Span-1093" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1094" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-1095" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1096" style="font-family: MathJax_Math-italic;">τ<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1097"><span class="mrow" id="MathJax-Span-1098"><span class="mi" id="MathJax-Span-1099" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1100" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1101" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1102" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mi" id="MathJax-Span-1103" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi></mrow></msub><mo>+</mo><msub><mi>p</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi></mrow></msub><mo>+</mo><msub><mi>τ</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>−</mo><mi>M</mi></math></span></span><script id="MathJax-Element-78" type="math/tex">t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M</script>. But <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-79-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1104" style="width: 1.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1001.01em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1105"><span class="mi" id="MathJax-Span-1106" style="font-family: MathJax_Math-italic;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>M</mi></math></span></span><script id="MathJax-Element-79" type="math/tex">M</script> is a very large number, then <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C4;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;mo&gt;&amp;lt;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-80-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1107" style="width: 11.372em; display: inline-block;"><span style="display: inline-block; position: relative; width: 9.467em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1009.41em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1108"><span class="msubsup" id="MathJax-Span-1109"><span style="display: inline-block; position: relative; width: 0.658em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1110" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-1111"><span class="mrow" id="MathJax-Span-1112"><span class="mi" id="MathJax-Span-1113" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1114" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-1115" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1116" style="font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1117"><span class="mrow" id="MathJax-Span-1118"><span class="mi" id="MathJax-Span-1119" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1120" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-1121" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 1.253em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1122" style="font-family: MathJax_Math-italic;">τ<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1123"><span class="mrow" id="MathJax-Span-1124"><span class="mi" id="MathJax-Span-1125" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1126" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1127" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1128" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mi" id="MathJax-Span-1129" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1130" style="font-family: MathJax_Main; padding-left: 0.301em;">&lt;</span><span class="mn" id="MathJax-Span-1131" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi></mrow></msub><mo>+</mo><msub><mi>p</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi></mrow></msub><mo>+</mo><msub><mi>τ</mi><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>−</mo><mi>M</mi><mo>&lt;</mo><mn>0</mn></math></span></span><script id="MathJax-Element-80" type="math/tex">t_{i} + p_{i} + \tau_{i,j} - M < 0</script> and 
since <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-81-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1132" style="width: 3.158em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.56em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1133"><span class="msubsup" id="MathJax-Span-1134"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1135" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-1136"><span class="mrow" id="MathJax-Span-1137"><span class="mi" id="MathJax-Span-1138" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1139" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-1140" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><mn>0</mn></math></span></span><script id="MathJax-Element-81" type="math/tex">t_{j} \geq 0</script>, this constraint is redundant.</p>
<ul>
<li><strong>Time window:</strong> For each job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-82-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1141" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1142"><span class="mi" id="MathJax-Span-1143" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1144" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1145" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-82" type="math/tex">j \in J</script> ensure that the time window for the job is satisfied.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-9"&gt;&lt;mtext&gt;(9)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-83-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1146" style="width: 100%; display: inline-block; min-width: 13.753em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 13.753em;"><span style="position: absolute; clip: rect(3.039em, 1009.59em, 4.467em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1147"><span class="mtable" id="MathJax-Span-1148" style="min-width: 13.753em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 13.753em;"><span style="display: inline-block; position: absolute; width: 9.586em; height: 0px; clip: rect(-0.949em, 1009.59em, 0.479em, -999.997em); top: 0em; left: 50%; margin-left: -4.818em;"><span style="position: absolute; clip: rect(3.098em, 1009.59em, 4.467em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 9.586em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1009.59em, 4.467em, -999.997em); top: -3.985em; left: 50%; margin-left: -4.818em;"><span class="mtd" id="MathJax-Span-1152"><span class="mrow" id="MathJax-Span-1153"><span class="msubsup" id="MathJax-Span-1154"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1155" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-1156"><span class="mrow" id="MathJax-Span-1157"><span class="mi" id="MathJax-Span-1158" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1159" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-1160" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1161" style="font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.539em;"><span class="texatom" id="MathJax-Span-1162"><span class="mrow" id="MathJax-Span-1163"><span class="mi" id="MathJax-Span-1164" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1165" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mi" id="MathJax-Span-1166" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">x</span><span class="msubsup" id="MathJax-Span-1167"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1168" style="font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.539em;"><span class="texatom" id="MathJax-Span-1169"><span class="mrow" id="MathJax-Span-1170"><span class="mi" id="MathJax-Span-1171" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1172" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1173" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1174" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1175" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1176" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-0.949em, 1001.19em, 0.42em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -3.985em; right: 0em;"><span class="mtd" id="mjx-eqn-9"><span class="mrow" id="MathJax-Span-1150"><span class="mtext" id="MathJax-Span-1151" style="font-family: MathJax_Main;">(9)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-9"><mtext>(9)</mtext></mtd><mtd><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><msub><mi>a</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>−</mo><mi>x</mi><msub><mi>a</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><mi>J</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-83" type="math/tex; mode=display">\begin{equation}
t_{j} \geq a_{j} - xa_{j} \quad \forall j \in J
\tag{9}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-10"&gt;&lt;mtext&gt;(10)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;b&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;b&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-84-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1177" style="width: 100%; display: inline-block; min-width: 14.586em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 14.586em;"><span style="position: absolute; clip: rect(3.039em, 1009.41em, 4.467em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1178"><span class="mtable" id="MathJax-Span-1179" style="min-width: 14.586em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 14.586em;"><span style="display: inline-block; position: absolute; width: 9.408em; height: 0px; clip: rect(-0.949em, 1009.41em, 0.479em, -999.997em); top: 0em; left: 50%; margin-left: -4.699em;"><span style="position: absolute; clip: rect(3.098em, 1009.41em, 4.467em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 9.408em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1009.41em, 4.467em, -999.997em); top: -3.985em; left: 50%; margin-left: -4.699em;"><span class="mtd" id="MathJax-Span-1183"><span class="mrow" id="MathJax-Span-1184"><span class="msubsup" id="MathJax-Span-1185"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1186" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-1187"><span class="mrow" id="MathJax-Span-1188"><span class="mi" id="MathJax-Span-1189" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1190" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-1191" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1192" style="font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1193"><span class="mrow" id="MathJax-Span-1194"><span class="mi" id="MathJax-Span-1195" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1196" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mi" id="MathJax-Span-1197" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">x</span><span class="msubsup" id="MathJax-Span-1198"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1199" style="font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1200"><span class="mrow" id="MathJax-Span-1201"><span class="mi" id="MathJax-Span-1202" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1203" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1204" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1205" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1206" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1207" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.789em; height: 0px; clip: rect(-0.949em, 1001.67em, 0.42em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.67em, 4.408em, -999.997em); top: -3.985em; right: 0em;"><span class="mtd" id="mjx-eqn-10"><span class="mrow" id="MathJax-Span-1181"><span class="mtext" id="MathJax-Span-1182" style="font-family: MathJax_Main;">(10)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-10"><mtext>(10)</mtext></mtd><mtd><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≤</mo><msub><mi>b</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>+</mo><mi>x</mi><msub><mi>b</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><mi>J</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-84" type="math/tex; mode=display">\begin{equation}
t_{j} \leq b_{j} + xb_{j} \quad \forall j \in J
\tag{10}
\end{equation}</script><p><strong>Note</strong>: To discourage that the time window of a job is violated, we associate the  penalty of 
(<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mn&gt;0.01&lt;/mn&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;&amp;#x03C0;&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mi&gt;M&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-85-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1208" style="width: 6.372em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.301em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1005.3em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1209"><span class="mn" id="MathJax-Span-1210" style="font-family: MathJax_Main;">0.01</span><span class="mo" id="MathJax-Span-1211" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1212" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.955em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.6em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1213" style="font-family: MathJax_Math-italic;">π<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.598em;"><span class="texatom" id="MathJax-Span-1214"><span class="mrow" id="MathJax-Span-1215"><span class="mi" id="MathJax-Span-1216" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1217" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mi" id="MathJax-Span-1218" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">M<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mn>0.01</mn><mo>⋅</mo><msub><mi>π</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>⋅</mo><mi>M</mi></math></span></span><script id="MathJax-Element-85" type="math/tex">0.01 \cdot \pi_{j} \cdot M</script>) to the correction variables <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;b&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-86-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1219" style="width: 3.932em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.277em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1003.28em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1220"><span class="mi" id="MathJax-Span-1221" style="font-family: MathJax_Math-italic;">x</span><span class="msubsup" id="MathJax-Span-1222"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1223" style="font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.539em;"><span class="texatom" id="MathJax-Span-1224"><span class="mrow" id="MathJax-Span-1225"><span class="mi" id="MathJax-Span-1226" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1227" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1228" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">x</span><span class="msubsup" id="MathJax-Span-1229"><span style="display: inline-block; position: relative; width: 0.777em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.42em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1230" style="font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.42em;"><span class="texatom" id="MathJax-Span-1231"><span class="mrow" id="MathJax-Span-1232"><span class="mi" id="MathJax-Span-1233" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>x</mi><msub><mi>a</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>,</mo><mi>x</mi><msub><mi>b</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-86" type="math/tex">xa_{j}, xb_{j}</script>.</p>
<ul>
<li><strong>Lateness constraint:</strong> For each job <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-87-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1234" style="width: 2.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.32em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1235"><span class="mi" id="MathJax-Span-1236" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1237" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1238" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi><mo>∈</mo><mi>J</mi></math></span></span><script id="MathJax-Element-87" type="math/tex">j \in J</script> calculate the lateness of the job.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-11"&gt;&lt;mtext&gt;(11)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;msub&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;msub&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;J&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-88-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1239" style="width: 100%; display: inline-block; min-width: 16.789em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 16.789em;"><span style="position: absolute; clip: rect(3.039em, 1011.61em, 4.467em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1240"><span class="mtable" id="MathJax-Span-1241" style="min-width: 16.789em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 16.789em;"><span style="display: inline-block; position: absolute; width: 11.61em; height: 0px; clip: rect(-0.949em, 1011.61em, 0.479em, -999.997em); top: 0em; left: 50%; margin-left: -5.83em;"><span style="position: absolute; clip: rect(3.098em, 1011.61em, 4.467em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 11.61em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1011.61em, 4.467em, -999.997em); top: -3.985em; left: 50%; margin-left: -5.83em;"><span class="mtd" id="MathJax-Span-1245"><span class="mrow" id="MathJax-Span-1246"><span class="msubsup" id="MathJax-Span-1247"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1248" style="font-family: MathJax_Math-italic;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1249"><span class="mrow" id="MathJax-Span-1250"><span class="mi" id="MathJax-Span-1251" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1252" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-1253" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.36em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1254" style="font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.36em;"><span class="texatom" id="MathJax-Span-1255"><span class="mrow" id="MathJax-Span-1256"><span class="mi" id="MathJax-Span-1257" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1258" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-1259" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1260" style="font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1261"><span class="mrow" id="MathJax-Span-1262"><span class="mi" id="MathJax-Span-1263" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1264" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mi" id="MathJax-Span-1265" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="msubsup" id="MathJax-Span-1266"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.54em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1267" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.539em;"><span class="texatom" id="MathJax-Span-1268"><span class="mrow" id="MathJax-Span-1269"><span class="mi" id="MathJax-Span-1270" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1271" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1272" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1273" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-1274" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1275" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">J<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.789em; height: 0px; clip: rect(-0.949em, 1001.67em, 0.42em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.67em, 4.408em, -999.997em); top: -3.985em; right: 0em;"><span class="mtd" id="mjx-eqn-11"><span class="mrow" id="MathJax-Span-1243"><span class="mtext" id="MathJax-Span-1244" style="font-family: MathJax_Main;">(11)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-11"><mtext>(11)</mtext></mtd><mtd><msub><mi>z</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>≥</mo><msub><mi>t</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>+</mo><msub><mi>p</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mo>−</mo><mi>d</mi><msub><mi>d</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>j</mi><mo>∈</mo><mi>J</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-88" type="math/tex; mode=display">\begin{equation}
z_{j} \geq t_{j} + p_{j} - dd_{j} \quad \forall j \in J
\tag{11}
\end{equation}</script><ul>
<li>Note that since the lateness decision variable <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mi&gt;z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-89-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1276" style="width: 1.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.84em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1277"><span class="msubsup" id="MathJax-Span-1278"><span style="display: inline-block; position: relative; width: 0.836em; height: 0px;"><span style="position: absolute; clip: rect(3.396em, 1000.48em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-1279" style="font-family: MathJax_Math-italic;">z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.479em;"><span class="texatom" id="MathJax-Span-1280"><span class="mrow" id="MathJax-Span-1281"><span class="mi" id="MathJax-Span-1282" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>z</mi><mrow class="MJX-TeXAtom-ORD"><mi>j</mi></mrow></msub></math></span></span><script id="MathJax-Element-89" type="math/tex">z_{j}</script> is non-negative, there is no benefit to complete a job before its due date; on the other hand, since the objective function minimizes the total weighted lateness, Constraint (11) should always be binding.</li>
</ul>



<h2 id="Problem-Instance">Problem Instance<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Problem-Instance">¶</a></h2><p>In this scenario, we consider the problem of routing and scheduling technicians for the next work-day in such a way that the delay of customer appointments is minimized. The telecom firm has seven technicians: Albert, Bob, Carlos, Doris, Ed, Flor, and Gina. There are two service centers: Heidelberg and Freiburg im Breisgau. Technicians are based at only one of these service centers. The number of hours available and the service center base (depot) for each technician is described in the following table.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Albert</th>
<th>Bob</th>
<th>Carlos</th>
<th>Doris</th>
<th>Ed</th>
<th>Flor</th>
<th>Gina</th>
</tr>
</thead>
<tbody>
<tr>
<td>Minutes</td>
<td>480</td>
<td>480</td>
<td>480</td>
<td>480</td>
<td>480</td>
<td>360</td>
<td>360</td>
</tr>
<tr>
<td>Depot</td>
<td>Heidelberg</td>
<td>Heidelberg</td>
<td>Freiburg im Breisgau</td>
<td>Freiburg im Breisgau</td>
<td>Heidelberg</td>
<td>Freiburg im Breisgau</td>
<td>Heidelberg</td>
</tr>
</tbody>
</table>
<p>The telecom company has different type of jobs. 
The following table shows the priority (4 for the highest priority and 1 for the least important) and duration (in hours) of a job type.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Priority</th>
<th>Duration (min)</th>
</tr>
</thead>
<tbody>
<tr>
<td>Equipment Installation</td>
<td>2</td>
<td>60</td>
</tr>
<tr>
<td>Equipment Setup</td>
<td>3</td>
<td>30</td>
</tr>
<tr>
<td>Inspect/Service Equipment</td>
<td>1</td>
<td>60</td>
</tr>
<tr>
<td>Repair - Regular</td>
<td>1</td>
<td>60</td>
</tr>
<tr>
<td>Repair - Important</td>
<td>2</td>
<td>120</td>
</tr>
<tr>
<td>Repair - Urgent</td>
<td>3</td>
<td>90</td>
</tr>
<tr>
<td>Repair - Critical</td>
<td>4</td>
<td>60</td>
</tr>
</tbody>
</table>
<p>The following table shows the jobs that each technician is qualified for.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Albert</th>
<th>Bob</th>
<th>Carlos</th>
<th>Doris</th>
<th>Ed</th>
<th>Flor</th>
<th>Gina</th>
</tr>
</thead>
<tbody>
<tr>
<td>Equipment Installation</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>1</td>
<td>-</td>
<td>1</td>
<td>1</td>
</tr>
<tr>
<td>Equipment Setup</td>
<td>1</td>
<td>1</td>
<td>1</td>
<td>1</td>
<td>-</td>
<td>-</td>
<td>1</td>
</tr>
<tr>
<td>Inspect/Service Equipment</td>
<td>1</td>
<td>-</td>
<td>1</td>
<td>-</td>
<td>1</td>
<td>-</td>
<td>-</td>
</tr>
<tr>
<td>Repair - Regular</td>
<td>-</td>
<td>1</td>
<td>1</td>
<td>-</td>
<td>1</td>
<td>1</td>
<td>1</td>
</tr>
<tr>
<td>Repair - Important</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>1</td>
<td>-</td>
<td>1</td>
<td>1</td>
</tr>
<tr>
<td>Repair - Urgent</td>
<td>-</td>
<td>1</td>
<td>1</td>
<td>-</td>
<td>1</td>
<td>1</td>
<td>1</td>
</tr>
<tr>
<td>Repair - Critical</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>1</td>
<td>-</td>
<td>-</td>
<td>1</td>
</tr>
</tbody>
</table>
<p>The telecom company receives customers' requests for a specific job type, appointment (due) time, and the service time window where the technician can arrive. The following table shows the customers' orders and their requirements. For each customer, the location is specified.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>C1:Mannheim</th>
<th>C2: Karlsruhe</th>
<th>C3: Baden-Baden</th>
<th>C4: Bühl</th>
<th>C5: Offenburg</th>
<th>C6: Lahr/Schwarzwald</th>
<th>C7: Lörrach</th>
</tr>
</thead>
<tbody>
<tr>
<td>Job type</td>
<td>Equipment Setup</td>
<td>Equipment Setup</td>
<td>Repair - Regular</td>
<td>Equipment Installation</td>
<td>Equipment Installation</td>
<td>Repair - Critical</td>
<td>Inspect/Service Equipment</td>
</tr>
<tr>
<td>Due time</td>
<td>8:00</td>
<td>10:00</td>
<td>11:00</td>
<td>12:00</td>
<td>14:00</td>
<td>15:00</td>
<td>16:00</td>
</tr>
<tr>
<td>Time Window</td>
<td>7:00-7:30</td>
<td>7:30-9:30</td>
<td>8:00-10:00</td>
<td>9:00-11:00</td>
<td>11:00-13:00</td>
<td>12:00-14:00</td>
<td>13:00-15:00</td>
</tr>
</tbody>
</table>
<p>The planning horizon is from 7:00 to 17:00, or 10 hours. The time period is in minutes, then the due time and time windows will be translated into minutes starting at 0 minutes and ending at 600 minutes. For example, for customer C2 the due time is at 10:00  (180 min), and the time window is from 7:30 to 9:30  (30 min to  150 min).</p>
<p>The following table shows the travel time (in minutes) from any depot or customer location to any depot or customer location.</p>
<table>
<thead><tr>
<th><i></i></th>
<th>Heidelberg</th>
<th>Freiburg im Breisgau</th>
<th>Mannheim</th>
<th>Karlsruhe</th>
<th>Baden-Baden</th>
<th>Bühl</th>
<th>Offenburg</th>
<th>Lahr/Schwarzwald</th>
<th>Lörrach</th>
</tr>
</thead>
<tbody>
<tr>
<td>Heidelberg</td>
<td>-</td>
<td>120</td>
<td>24</td>
<td>50</td>
<td>67</td>
<td>71</td>
<td>88</td>
<td>98</td>
<td>150</td>
</tr>
<tr>
<td>Freiburg im Breisgau</td>
<td>-</td>
<td>-</td>
<td>125</td>
<td>85</td>
<td>68</td>
<td>62</td>
<td>45</td>
<td>39</td>
<td>48</td>
</tr>
<tr>
<td>Mannheim</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>53</td>
<td>74</td>
<td>77</td>
<td>95</td>
<td>106</td>
<td>160</td>
</tr>
<tr>
<td>Karlsruhe</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>31</td>
<td>35</td>
<td>51</td>
<td>61</td>
<td>115</td>
</tr>
<tr>
<td>Baden-Baden</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>16</td>
<td>36</td>
<td>46</td>
<td>98</td>
</tr>
<tr>
<td>Bühl</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>30</td>
<td>40</td>
<td>92</td>
</tr>
<tr>
<td>Offenburg</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>26</td>
<td>80</td>
</tr>
<tr>
<td>Lahr/Schwarzwald</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>70</td>
</tr>
<tr>
<td>Lörrach</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
</tr>
</tbody>
</table>



<h2 id="Python-Implementation">Python Implementation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Python-Implementation">¶</a></h2><p>In this Jupyter Notebook, we use the following libraries:</p>
<ul>
<li><code>sys</code> to access system-specific parameters and functions.</li>
<li><code>defaultdict</code> to supply missing values for arbitrary keys.</li>
<li><code>xlrd</code> to read data from Excel files.</li>
<li><code>gurobipy</code> Gurobi Optimizer library.</li>
</ul>
<p>This implementation is based on object-oriented-programming.</p>
<h3 id="Helper-Classes">Helper Classes<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Helper-Classes">¶</a></h3><p>The following classes properly organize the input data of the MIP model.</p>
<ul>
<li><code>Technician</code>.</li>
<li><code>Job</code>.</li>
<li><code>Customer</code>.</li>
</ul>
<h3 id="Helper-Functions">Helper Functions<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Helper-Functions">¶</a></h3><ul>
<li><code>solve_trs0</code> builds and solves MIP model.</li>
<li><code>printScen</code> prints headings for output reports.</li>
</ul>


In [ ]:

import sys
from collections import defaultdict
import xlrd
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0



In [ ]:

class Technician():
    def __init__(self, name, cap, depot):
        self.name = name
        self.cap = cap
        self.depot = depot

    def __str__(self):
        return f"Technician: {self.name}\n  Capacity: {self.cap}\n  Depot: {self.depot}"



In [ ]:

class Job():
    def __init__(self, name, priority, duration, coveredBy):
        self.name = name
        self.priority = priority
        self.duration = duration
        self.coveredBy = coveredBy

    def __str__(self):
        about = f"Job: {self.name}\n  Priority: {self.priority}\n  Duration: {self.duration}\n  Covered by: "
        about += ", ".join([t.name for t in self.coveredBy])
        return about



In [ ]:

class Customer():
    def __init__(self, name, loc, job, tStart, tEnd, tDue):
        self.name = name
        self.loc = loc
        self.job = job
        self.tStart = tStart
        self.tEnd = tEnd
        self.tDue = tDue

    def __str__(self):
        coveredBy = ", ".join([t.name for t in self.job.coveredBy])
        return f"Customer: {self.name}\n  Location: {self.loc}\n  Job: {self.job.name}\n  Priority: {self.job.priority}\n  Duration: {self.job.duration}\n  Covered by: {coveredBy}\n  Start time: {self.tStart}\n  End time: {self.tEnd}\n  Due time: {self.tDue}"




<h2 id="Base-Scenario">Base Scenario<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Base-Scenario">¶</a></h2><p>For the base scenario, there is enough technician capacity to fill all customers demand. To run the base scenario, consider the spreadsheet file <code>data-Sce0.xlsx</code> and insert this name when you open the Excel workbook.</p>


In [ ]:

# Open Excel workbook
wb = xlrd.open_workbook('data-Sce0.xlsx')

# Read technician data
ws = wb.sheet_by_name('Technicians')
technicians = []
for i,t in enumerate(ws.col_values(0)[3:]):
    # Create Technician object
    thisTech = Technician(*ws.row_values(3+i)[:3])
    technicians.append(thisTech)



In [ ]:

# Read job data
jobs = []
for j,b in enumerate(ws.row_values(0)[3:]):
    coveredBy = [t for i,t in enumerate(technicians) if ws.cell_value(3+i,3+j) == 1]
    # Create Job object
    thisJob = Job(*ws.col_values(3+j)[:3], coveredBy)
    jobs.append(thisJob)



In [ ]:

# Read location data
ws = wb.sheet_by_name('Locations')
locations = ws.col_values(0)[1:]
dist = {(l, l) : 0 for l in locations}
for i,l1 in enumerate(locations):
    for j,l2 in enumerate(locations):
        if i < j:
            dist[l1,l2] = ws.cell_value(1+i, 1+j)
            dist[l2,l1] = dist[l1,l2]



In [ ]:

# Read customer data
ws = wb.sheet_by_name('Customers')
customers = []
for i,c in enumerate(ws.col_values(0)[1:]):
    for b in jobs:
        if b.name == ws.cell_value(1+i, 2):
            # Create Customer object using corresponding Job object
            rowVals = ws.row_values(1+i)
            #print(rowVals)
            thisCustomer = Customer(*rowVals[:2], b, *rowVals[3:])
            customers.append(thisCustomer)
            break



In [ ]:

def solve_trs0(technicians, customers, dist):
    # Build useful data structures
    K = [k.name for k in technicians]
    C = [j.name for j in customers]
    J = [j.loc for j in customers]
    L = list(set([l[0] for l in dist.keys()]))
    D = list(set([t.depot for t in technicians]))
    cap = {k.name : k.cap for k in technicians}
    loc = {j.name : j.loc for j in customers}
    depot = {k.name : k.depot for k in technicians}
    canCover = {j.name : [k.name for k in j.job.coveredBy] for j in customers}
    dur = {j.name : j.job.duration for j in customers}
    tStart = {j.name : j.tStart for j in customers}
    tEnd = {j.name : j.tEnd for j in customers}
    tDue = {j.name : j.tDue for j in customers}
    priority = {j.name : j.job.priority for j in customers}
    
        ### Create model
    m = gp.Model("trs0")
    
    ### Decision variables
    # Customer-technician assignment
    x = m.addVars(C, K, vtype=GRB.BINARY, name="x")
    
    # Technician assignment
    u = m.addVars(K, vtype=GRB.BINARY, name="u")
    
    # Edge-route assignment to technician
    y = m.addVars(L, L, K, vtype=GRB.BINARY, name="y")
   
    # Technician cannot leave or return to a depot that is not its base
    for k in technicians:
        for d in D:
            if k.depot != d:
                for i in L:
                    y[i,d,k.name].ub = 0
                    y[d,i,k.name].ub = 0
    
    # Start time of service
    t = m.addVars(L, ub=600, name="t")
    
    # Lateness of service
    z = m.addVars(C, name="z")
    
    # Artificial variables to correct time window upper and lower limits
    xa = m.addVars(C, name="xa")
    xb = m.addVars(C, name="xb")
    
    # Unfilled jobs
    g = m.addVars(C, vtype=GRB.BINARY, name="g")
    
        ### Constraints

    # A technician must be assigned to a job, or a gap is declared (1)
    m.addConstrs((gp.quicksum(x[j, k] for k in canCover[j]) + g[j] == 1 for j in C), name="assignToJob")
    
    # At most one technician can be assigned to a job (2)
    m.addConstrs((x.sum(j, '*') <= 1 for j in C), name="assignOne")

    # Technician capacity constraints (3)
    capLHS = {k : gp.quicksum(dur[j]*x[j,k] for j in C) +\
        gp.quicksum(dist[i,j]*y[i,j,k] for i in L for j in L) for k in K}
    m.addConstrs((capLHS[k] <= cap[k]*u[k] for k in K), name="techCapacity")

    # Technician tour constraints (4 and 5)
    m.addConstrs((y.sum('*', loc[j], k) == x[j,k] for k in K for j in C),\
        name="techTour1")
    m.addConstrs((y.sum(loc[j], '*', k) == x[j,k] for k in K for j in C),\
        name="techTour2")

    # Same depot constraints (6 and 7)
    m.addConstrs((gp.quicksum(y[j,depot[k],k] for j in J) == u[k] for k in K),\
        name="sameDepot1")
    m.addConstrs((gp.quicksum(y[depot[k],j,k] for j in J) == u[k] for k in K),\
        name="sameDepot2")

    # Temporal constraints (8) for customer locations
    M = {(i,j) : 600 + dur[i] + dist[loc[i], loc[j]] for i in C for j in C}
    m.addConstrs((t[loc[j]] >= t[loc[i]] + dur[i] + dist[loc[i], loc[j]]\
        - M[i,j]*(1 - gp.quicksum(y[loc[i],loc[j],k] for k in K))\
        for i in C for j in C), name="tempoCustomer")

    # Temporal constraints (8) for depot locations
    M = {(i,j) : 600 + dist[i, loc[j]] for i in D for j in C}
    m.addConstrs((t[loc[j]] >= t[i] + dist[i, loc[j]]\
        - M[i,j]*(1 - y.sum(i,loc[j],'*')) for i in D for j in C),\
        name="tempoDepot")

    # Time window constraints (9 and 10)
    m.addConstrs((t[loc[j]] + xa[j] >= tStart[j] for j in C), name="timeWinA")
    m.addConstrs((t[loc[j]] - xb[j] <= tEnd[j] for j in C), name="timeWinB")

    # Lateness constraint (11)
    m.addConstrs((z[j] >= t[loc[j]] + dur[j] - tDue[j] for j in C),\
        name="lateness")

    ### Objective function
    M = 6100
    
    m.setObjective(z.prod(priority) + gp.quicksum( 0.01 * M * priority[j] * (xa[j] + xb[j]) for j in C) +
                   gp.quicksum( M * priority[j] * g[j] for j in C) , GRB.MINIMIZE)
    
    m.write("TRS0.lp")
    m.optimize()

    status = m.Status
    if status in [GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED]:
        print("Model is either infeasible or unbounded.")
        sys.exit(0)
    elif status != GRB.OPTIMAL:
        print("Optimization terminated with status {}".format(status))
        sys.exit(0)
        
    ### Print results
    # Assignments
    print("")
    for j in customers:
        if g[j.name].X > 0.5:
            jobStr = "Nobody assigned to {} ({}) in {}".format(j.name,j.job.name,j.loc)
        else:
            for k in K:
                if x[j.name,k].X > 0.5:
                    jobStr = "{} assigned to {} ({}) in {}. Start at t={:.2f}.".format(k,j.name,j.job.name,j.loc,t[j.loc].X)
                    if z[j.name].X > 1e-6:
                        jobStr += " {:.2f} minutes late.".format(z[j.name].X)
                    if xa[j.name].X > 1e-6:
                        jobStr += " Start time corrected by {:.2f} minutes.".format(xa[j.name].X)
                    if xb[j.name].X > 1e-6:
                        jobStr += " End time corrected by {:.2f} minutes.".format(xb[j.name].X)
        print(jobStr)

    # Technicians
    print("")
    for k in technicians:
        if u[k.name].X > 0.5:
            cur = k.depot
            route = k.depot
            while True:
                for j in customers:
                    if y[cur,j.loc,k.name].X > 0.5:
                        route += " -> {} (dist={}, t={:.2f}, proc={})".format(j.loc, dist[cur,j.loc], t[j.loc].X, j.job.duration)
                        cur = j.loc
                for i in D:
                    if y[cur,i,k.name].X > 0.5:
                        route += " -> {} (dist={})".format(i, dist[cur,i])
                        cur = i
                        break
                if cur == k.depot:
                    break
            print("{}'s route: {}".format(k.name, route))
        else:
            print("{} is not used".format(k.name)) 
            
    
    # Utilization
    print("")
    for k in K:
        used = capLHS[k].getValue()
        total = cap[k]
        util = used / cap[k] if cap[k] > 0 else 0
        print("{}'s utilization is {:.2%} ({:.2f}/{:.2f})".format(k, util,\
            used, cap[k]))
    totUsed = sum(capLHS[k].getValue() for k in K)
    totCap = sum(cap[k] for k in K)
    totUtil = totUsed / totCap if totCap > 0 else 0
    print("Total technician utilization is {:.2%} ({:.2f}/{:.2f})".format(totUtil, totUsed, totCap))
    
    
    



In [ ]:

def printScen(scenStr):
    sLen = len(scenStr)
    print("\n" + "*"*sLen + "\n" + scenStr + "\n" + "*"*sLen + "\n")



In [ ]:

if __name__ == "__main__":
    # Base model
    printScen("Solving base scenario model")
    solve_trs0(technicians, customers, dist)




<h2 id="Base-Scenario-Analysis">Base Scenario Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Base-Scenario-Analysis">¶</a></h2><p>For the base scenario, we have enough technician capacity to satisfy customers demand. Notice that the optimal objective function value is 0.00, this means that all jobs were completed, they were completed on time, and there were no corrections to the time window limits.</p>
<p>The first report describes which technician was assigned to each customer, the location of the customer, and the start time of the job. The second report describes the route of each assigned technician. The route defines the <em>from-location</em> and the 
<em>to-location</em>. The parameter <em>dist</em>  defines the number of minutes that it takes to drive from the <em>from-location</em> to the <em>to-location</em>. The parameter t determines the <em>start-time</em> of the job at the <em>to-location</em>. The parameter <em>proc</em>  shows the number of minutes that it takes the technician to complete the job. 
The third report describes the capacity utilization of each technician and the overall capacity utilization of all technicians. The technician capacity utilization is the number of minutes the technician spends driving or serving customers, divided by the technician capacity. The overall capacity utilization of all technicians is the total number of minutes that all technicians spend driving or serving customers, divided by the total capacity of all the technicians available.</p>



<h2 id="Scenario-1">Scenario 1<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Scenario-1">¶</a></h2><p>We assume that technicians are working at half of their capacity. Also, we assume that customer 5 requires an urgent service and its due time is at 00 min. Since the due time has changed, we remove the time window limits, that is the time window is any time within the planning horizon. The new time window is [0, 600].</p>
<p>To run this scenario, consider the spreadsheet file <code>data-Sce3.xlsx</code> and insert this name when you open the Excel workbook.</p>


In [ ]:

# Open Excel workbook
wb = xlrd.open_workbook('data-Sce3.xlsx')

# Read technician data
ws = wb.sheet_by_name('Technicians')
technicians = []
for i,t in enumerate(ws.col_values(0)[3:]):
    # Create Technician object
    thisTech = Technician(*ws.row_values(3+i)[:3])
    technicians.append(thisTech)



In [ ]:

# Read job data
jobs = []
for j,b in enumerate(ws.row_values(0)[3:]):
    coveredBy = [t for i,t in enumerate(technicians) if ws.cell_value(3+i,3+j) == 1]
    # Create Job object
    thisJob = Job(*ws.col_values(3+j)[:3], coveredBy)
    jobs.append(thisJob)



In [ ]:

# Read location data
ws = wb.sheet_by_name('Locations')
locations = ws.col_values(0)[1:]
dist = {(l, l) : 0 for l in locations}
for i,l1 in enumerate(locations):
    for j,l2 in enumerate(locations):
        if i < j:
            dist[l1,l2] = ws.cell_value(1+i, 1+j)
            dist[l2,l1] = dist[l1,l2]



In [ ]:

# Read customer data
ws = wb.sheet_by_name('Customers')
customers = []
for i,c in enumerate(ws.col_values(0)[1:]):
    for b in jobs:
        if b.name == ws.cell_value(1+i, 2):
            # Create Customer object using corresponding Job object
            rowVals = ws.row_values(1+i)
            #print(rowVals)
            thisCustomer = Customer(*rowVals[:2], b, *rowVals[3:])
            customers.append(thisCustomer)
            break



In [ ]:

if __name__ == "__main__":
    # Base model
    printScen("Solving scenario 1 model")
    solve_trs0(technicians, customers, dist)




<h2 id="Scenario-1-Analysis">Scenario 1 Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Scenario-1-Analysis">¶</a></h2><p>We assume that technicians are working at half of their capacity. Also, we assume that customer 5 requires an urgent service and its due time is at 00 min. Since the due time has changed, we remove the time window limits, that is the time window is any time within the planning horizon. The new time window is [0, 600].</p>
<ul>
<li>Six jobs out of seven are filled.<ul>
<li>The service of customer 5 will be 105 min late. </li>
<li>The start time is at min 45 which is within the time window of [0, 600].</li>
</ul>
</li>
<li>Service requirement of Customer 4 could not be filled. <ul>
<li>Observe that the priority of this customer is low.</li>
</ul>
</li>
<li>Six technicians were assigned to fill the requirements of 6 customers.</li>
<li>Ed was not assigned to any customer. <ul>
<li>Notice that Ed is not qualified for the job of Customer 4 (Equipment Installation).</li>
</ul>
</li>
<li>The overall technician utilization is 54.23%.</li>
</ul>



<h2 id="Conclusion">Conclusion<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#Conclusion">¶</a></h2><p>In this Jupyter Notebook, we formulated and solved a multi-depot vehicle routing problem with time windows constraints using the Gurobi Python API. This MIP formulation is an optimization application for the telecommunication industry; however, the formulation is quite general and can easily be adapted to any kind of vehicle routing problems in the transportation and logistics industries.</p>
<p>The routing and scheduling problem that we tackled considers a telecom firm that operates multiple service centers to serve its customers. We solved this routing and scheduling problem by simultaneously making three types of decisions:</p>
<ul>
<li>(i) the assignment of jobs to a technician at all the service centers</li>
<li>(ii) the routing of each technician, i.e. the sequence/order of customers for a technician to visit</li>
<li>(iii) the scheduling of jobs, i.e. the timing for a technician to arrive at a customer location and complete the corresponding job. </li>
</ul>
<p>The objective of the telecom firm is to minimize the total weighted lateness of all the jobs.</p>
<p>In this notebook, we discussed two scenarios, the base scenario has enough technicians' capacity to fill all customers demand on time. In scenario 1, we reduced the capacity of all technicians by half, and one customer has an emergency that requires service first thing in the morning. The scenario captures various trade-offs that the MIP model needs to consider given the limited technician capacity and customers' job priority and requirements.</p>



<h2 id="References">References<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html#References">¶</a></h2><p>[1] S. Salhi, A. Imran, N. A. Wassan. <em>The multi-depot vehicle routing problem with heterogeneous vehicle fleet: Formulation and a variable neighborhood search implementation</em>. Computers &amp; Operations Research 52 (2014) 315-325.</p>



<p>Copyright © 2020 Gurobi Optimization, LLC</p>
